# Успешен експеримент

Тренираме модела с 22715 тренировъчни и 7587 валидационни примера. Пуснах тренирането със следните параметри в anchor_parameters.ini:

Разликите спрямо параметрите по подразбиране са:
- добавих ratio 0.2 с цел да хващат по-дългите кораби
- добавих scale 0.5 и премахнах 1.5874010519681994 с цел да се хващат по-малките кораби

Планът ми беше да го пусна с 20 епохи, но при при 15 вече се виждаше много малко подобрение на епоха (~0.005 mAP) и реших да го спра. Може би с повече данни ще има смисъл и от повече епохи.

In [ ]:
import keras

import keras_retinanet.models as models

from train import get_session, create_generators, create_models, create_callbacks, read_config_file

import warnings
warnings.simplefilter("ignore", UserWarning)


config = {
    'backbone': 'resnet50',
    'weights': 'keras-retinanet/snapshots/resnet50_coco_best_v2.1.0.h5',
    'dataset_type': 'csv',
    'annotations': '../data/train_v2/train.csv',
    'classes': '../data/bboxes_classes.csv',
    'val_annotations': '../data/train_v2/validation.csv',
    'multi_gpu': 0,
    'freeze_backbone': False,
    'lr': 1e-5,
    'config': './anchor_parameters.ini',
    
    'steps': None,
    'epochs': 20,
    'workers': 4,
    'max_queue_size': 10,
    'random_transform': False,
    
    'batch_size': 7,
    'image_min_side': 768,
    'image_max_side': 768,
    'snapshots': True,
    'snapshot_path': './snapshots',
    'evaluation': True,
    'weighted_average': False,
    'tensorboard_dir': './tensorboard_logs'
}

# create object that stores backbone information
backbone = models.backbone(config['backbone'])
keras.backend.tensorflow_backend.set_session(get_session())

if config['config']:
    config['config'] = read_config_file(config['config'])

# create the generators
train_generator, validation_generator = create_generators(config, backbone.preprocess_image)

print('Creating model, this may take a second...')
model, training_model, prediction_model = create_models(
    backbone_retinanet=backbone.retinanet,
    num_classes=train_generator.num_classes(),
    weights=config['weights'],
    multi_gpu=config['multi_gpu'],
    freeze_backbone=config['freeze_backbone'],
    lr=config['lr'],
    config=config['config']
)

print(model.summary())

# create the callbacks
callbacks = create_callbacks(
    model,
    training_model,
    prediction_model,
    validation_generator,
    config,
)

# start training
training_model.fit_generator(
    generator=train_generator,
    steps_per_epoch=config['steps'],
    epochs=config['epochs'],
    verbose=1,
    callbacks=callbacks,
    workers=config['workers'],
    use_multiprocessing=config['workers'] > 0,
    max_queue_size=config['max_queue_size']
)


Using TensorFlow backend.


Creating model, this may take a second...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
padding_conv1 (ZeroPadding2D)   (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9408        padding_conv1[0][0]              
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
___________________________________________________________________

Epoch 1/20
1128/1128 [==============================] - 3242s 3s/step - loss: 2.0591 - regression_loss: 1.6351 - classification_loss: 0.4240


Running network: 100% (2632 of 2632) |###| Elapsed Time: 0:08:16 Time:  0:08:16
Parsing annotations: 100% (2632 of 2632) || Elapsed Time: 0:00:00 Time:  0:00:00


7254 instances of class ship with average precision: 0.5214
mAP: 0.5214

Epoch 00001: saving model to ./snapshots/resnet50_csv_01.h5
Epoch 2/20
1128/1128 [==============================] - 3214s 3s/step - loss: 1.1936 - regression_loss: 0.9777 - classification_loss: 0.2159


...

Running network: 100% (2632 of 2632) |###| Elapsed Time: 0:08:14 Time:  0:08:14
Parsing annotations: 100% (2632 of 2632) || Elapsed Time: 0:00:00 Time:  0:00:00


7254 instances of class ship with average precision: 0.7265
mAP: 0.7265

Epoch 00013: saving model to ./snapshots/resnet50_csv_13.h5
Epoch 14/20
1128/1128 [==============================] - 3217s 3s/step - loss: 0.3545 - regression_loss: 0.3050 - classification_loss: 0.0494


Running network: 100% (2632 of 2632) |###| Elapsed Time: 0:08:12 Time:  0:08:12
Parsing annotations: 100% (2632 of 2632) || Elapsed Time: 0:00:00 Time:  0:00:00


7254 instances of class ship with average precision: 0.7312
mAP: 0.7312

Epoch 00014: saving model to ./snapshots/resnet50_csv_14.h5
Epoch 15/20
1128/1128 [==============================] - 3216s 3s/step - loss: 0.3314 - regression_loss: 0.2858 - classification_loss: 0.0456


Running network: 100% (2632 of 2632) |###| Elapsed Time: 0:08:13 Time:  0:08:13
Parsing annotations: 100% (2632 of 2632) || Elapsed Time: 0:00:00 Time:  0:00:00


Тренирането отнемаше по около час на епоха. Т.е. за около 15 часа постигнахме mAP ~ 0.73